In [33]:
from siamese_dataloader import Siamese_dataset_folder, Siamese_dataset, Siamese_dataloader
from torch import nn as nn
from torch import device, cuda, no_grad, manual_seed, cat
import torch.optim as optim
from torch.utils.data import Subset
from numpy import load, logical_not, logical_and, arange
from tqdm import tqdm
import numpy as np
import torch

In [2]:
SEED = 2024

root="../train_xml"
# Raw data
subset = logical_not(load("data/missing_file_names_mask.npy")) # Makes losses adapt to the selected mouse_records
losses = load("data/resnet50_losses_final_weights.npy")[subset]
mouse_records = load("data/mouse_record_interpolated.npy")

# Clean data aka reasonable estimated time
estimate_times=load('data/sample_estimate_times.npy')
clean_subset = estimate_times[subset] < 3000
clean_subset = logical_and(clean_subset, estimate_times[subset] > 0)

losses = losses[clean_subset]
mouse_records = mouse_records[clean_subset]

# Train and test split
assert len(losses) == len(mouse_records)
indices = list(arange(len(losses)))
indices_randomized = np.random.choice(indices, int(len(indices)), replace=False)
train_indices = indices_randomized[:int(0.8*len(indices))]
test_indices = indices_randomized[int(0.8*len(indices)):]

losses_train = losses[train_indices]
mouse_records_train = mouse_records[train_indices]
dataset_train = Siamese_dataset(losses_train, mouse_records_train, seed=SEED)
dataloader_train = Siamese_dataloader(dataset_train, batch_size=4096, num_workers=8, shuffle=True).run()


losses_test = losses[test_indices]
mouse_records_test = mouse_records[test_indices]
dataset_test = Siamese_dataset(losses_test, mouse_records_test, seed=SEED)
dataloader_test = Siamese_dataloader(dataset_test, batch_size=4096, num_workers=8, shuffle=True).run()



len(dataset_train), len(dataset_test)



(872249, 218063)

In [3]:
def train_one_epoch(train_loader, net, criterion, optimizer, epoch):
    running_loss = 0.0
    for _, (inputs_0, inputs_1, labels) in enumerate(train_loader, 0):
        inputs_0, inputs_1, labels = inputs_0.to(device), inputs_1.to(device), labels.to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs_0, inputs_1)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    return running_loss

In [6]:
class Siamese_network(nn.Module):
    # It should predict 0, 1.
    # 0 if 1st sample has higher loss
    # 1 if 2nd sample has higher loss
    def __init__(self):
        super().__init__()

        # input is 30x3 dim and output is 1 dim
        self.shared_features = nn.Sequential(
            nn.Linear(90, 128),
            nn.ReLU(),
            nn.BatchNorm1d(128),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.BatchNorm1d(32),
        )

        self.classification_head = nn.Sequential(
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.BatchNorm1d(16),
            nn.Linear(16, 4),
            nn.ReLU(),
            nn.BatchNorm1d(4),
            nn.Linear(4, 1),
        )

    def forward(self, x, y):
        x = self.shared_features(x)
        y = self.shared_features(y)

        return self.classification_head(x * y)


In [35]:
device = 'cuda:0' if cuda.is_available() else 'cpu'
print(f"Using device: {device}")

net = Siamese_network()
print(f"The number of parameters is {sum(p.numel() for p in net.parameters())}")
net.to(device)

# criterion = nn.BCEWithLogitsLoss()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(net.parameters(), lr=0.0001, momentum=0.8)

print("Training")
history = []

with no_grad():
    loss = 0
    accuracy = 0
    predictions = []
    all_labels = []
    for _, (inputs_0, inputs_1, labels) in enumerate(dataloader_test, 0):
        inputs_0, inputs_1, labels = inputs_0.to(device), inputs_1.to(device), labels.to(device)

        outputs = net(inputs_0, inputs_1)
        predictions.append(outputs)
        all_labels.append(labels)
        loss += criterion(outputs, labels).item()

    predictions = cat(predictions)
    all_labels = cat(all_labels)
preds = predictions<0
accuracy = (preds == all_labels).sum().item() / len(predictions)
print(f"Initial test accuracy is {accuracy} and loss is {loss}")

Using device: cpu
The number of parameters is 23073
Training
Initial test accuracy is 0.4999289196241453 and loss is 38.65213871002197


In [36]:
EPOCHS = 5
for _, epoch in enumerate(tqdm(range(EPOCHS))):
    e_loss = train_one_epoch(dataloader_train, net, criterion, optimizer, epoch)
    print(f"Epoch: {epoch}/{EPOCHS} | Loss : {e_loss}")
    history.append(e_loss)

with no_grad():
    loss = 0
    accuracy = 0
    predictions = []
    all_labels = []
    for _, (inputs_0, inputs_1, labels) in enumerate(dataloader_test, 0):
        inputs_0, inputs_1, labels = inputs_0.to(device), inputs_1.to(device), labels.to(device)
        outputs = net(inputs_0, inputs_1)
        predictions.append(outputs)
        all_labels.append(labels)
        loss += criterion(outputs, labels).item()

    predictions = cat(predictions)
    all_labels = cat(all_labels)
    preds = predictions<0
    accuracy = (preds == all_labels).sum().item() / len(predictions)
print(f"Finished Training, final accuracy is {accuracy} and loss is {loss}")


 20%|██        | 1/5 [00:20<01:20, 20.15s/it]

Epoch: 0/5 | Loss : 152.05445128679276


 40%|████      | 2/5 [00:35<00:52, 17.34s/it]

Epoch: 1/5 | Loss : 151.70633572340012


 60%|██████    | 3/5 [00:52<00:34, 17.17s/it]

...

 80%|████████  | 4/5 [01:12<00:18, 18.17s/it]

Epoch: 3/5 | Loss : 151.44574791193008


100%|██████████| 5/5 [01:31<00:00, 18.30s/it]

Epoch: 4/5 | Loss : 151.40130192041397


Finished Training, final accuracy is 0.5001903119740625 and loss is 38.43146401643753
